In [1]:
from varicirc import *
from measure import *
from scipy.optimize import minimize
from time import time

from qutip.random_objects import rand_ket
from qutip.qip.qubits import qubit_states
from qutip.tensor import tensor

In [17]:
N = 4 # number of qubits
n = 2 # number of reduced qubits

# Random inputs (qubit state)
Input = rand_ket(2**N,dims=[[2]*N,[1]*N])
# Reference State
r_state = qubit_states(n)

# Set the variational circuit
vc1 = vcirc(N)
vc11 = vcirc(n)
vc2 = vcirc(N)

def test_ptest(x):
    vc1.update_ansatzes(x)
    return ptest(Input,vc1.qc,parti=[[0,1],[2,3]])

def test_qdr(x):
    vc2.update_ansatzes(x)
    return qdr_cost(state2,r_state,vc2.qc,ref_sys=[2,3])

def test_phase2(x):
    vc11.update_ansatzes(x)
    return qdr_cost(Input,r_state,vc11.qc)

print("disentangling")

for L in np.arange(1,5):    # number of ansatz
    vc1.add_ansatz(np.zeros(N*3)) # add one layer
    t0 = time()
    x0 = np.zeros(N*3*L) # init parameters

    res = minimize(test_ptest,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break
    
print("The second phase")

prop = gate_sequence_product(vc1.qc.propagators())
out = prop*Input
state2 = out.ptrace([2,3])

for L in np.arange(1,5):
    vc11.add_ansatz(np.zeros(n*3)) # add one layer
    t0 = time()
    x0 = np.zeros(n*3*L) # init parameters

    res = minimize(test_phase2,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break

prop2 = gate_sequence_product(vc11.qc.propagators())
out2 = prop2*state2*prop2.dag()
final = tensor(out.ptrace([0,1]),out2)

print("result fidelity is",fidelity(final,tensor(out.ptrace([0,1]),r_state*r_state.dag())))

print("\noriginal dimensionality reduction")

for L in np.arange(1,5):    # number of ansatz
    vc2.add_ansatz(np.zeros(N*3)) # add one layer
    t0 = time()
    x0 = np.zeros(N*3*L) # init parameters

    res = minimize(test_qdr,x0,method='powell')
    print("L=",L,"Result=",res.fun,"Iteration time=",res.nit,"Time=",time()-t0,"seconds")
    if res.fun < 1e-5:
        break
prop = gate_sequence_product(vc2.qc.propagators())
out = prop*Input

print("result fidelity is",fidelity(tensor(out.ptrace([0,1]),r_state*r_state.dag()),out))

disentangling
L= 1 Result= 0.38957129808327917 Time= 24.283876180648804 seconds
L= 2 Result= 0.005253735848269714 Time= 265.5920798778534 seconds
L= 3 Result= 6.994405055138486e-15 Time= 569.9105548858643 seconds
The second phase
L= 1 Result= 0.08450330576163956 Time= 2.2333757877349854 seconds
L= 2 Result= 9.992007221626409e-16 Time= 27.207873582839966 seconds
result fidelity is 1.000000026225639

original dimensionality reduction
L= 1 Result= 0.2554105484410114 Time= 8.945960521697998 seconds
L= 2 Result= 0.06295969311283 Time= 68.74939727783203 seconds
L= 3 Result= 0.00021468552947123687 Time= 575.0494072437286 seconds
L= 4 Result= 2.220446049250313e-15 Time= 1076.0109598636627 seconds
result fidelity is 1.0000000183711755
